***This notebook contains a script that sends Student Contracts to Students for signing***

**Add local library to path**

In [4]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

***Import Libraries***

In [5]:
import pandas as pd
import requests
import json
from datetime import date
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [6]:
#import 2020 data worksheet
cohort_df = import_worksheet("Cohort 2020 Data", "Cohort 2020 Data")
cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']

#import assessor database
assessor_df = import_worksheet("SETA Assessor Registration Form (Responses)", "Form Responses 1")

**Create Learner FullName**

In [7]:
cohort_df['learner_fullname'] = cohort_df['Firstname'] + ' ' + cohort_df['Surname']

**Create Learner Initials**

In [8]:
cohort_df["learner_initials"] = [name.split()[0][0] + "." + name.split()[1][0] for name in cohort_df['Firstname'] + ' ' + cohort_df['Surname']]


**Create Learner Passport**

In [9]:
cohort_df["learner_passport"] = [passport if len(str(passport)) != 13 else " " for passport in cohort_df["ID Number/ Passport Number"]]


**Create Learner Equity Code**

In [10]:
cohort_df['learner_equity'] = cohort_df['Ethnicity'].map({'White':'WH', 'white':'WH', 'Coloured':'BC', 'coloured':'BC', 
                                                     'black':'BA', 'Black':'BA', 'Asian':'BI', 'indian':'BI','Indian':'BI'})

**Create Learner Learnership**

In [11]:
cohort_df['learner_nationality'] = cohort_df['Nationality'].map({'South African':'SA', 'Zimbabwean':'ZIM', 'Congolese':'ROA',
                                                                'Dutch':'EUR', 'Nigerian':'ROA'})

**Create Learner Citizenship**

In [12]:
cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'] = cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'].fillna('Uknown')
cohort_df['learner_citizenship'] = cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'].map({"Unknown":"U", "Citizen":"SA",
                                                                                  "Permanent Resident":"O", "Dual":"D"})

**Create Learner Langauge**

In [13]:
cohort_df['learner_language'] = cohort_df['Home Language'].map({'English':'ENG', 'Afrikaans':'AFR', 'Sepedi [also known as Northern Sotho / Sesotho sa Lebowa]':'SEP',
       'tshiVenda':'TSH', 'isiZulu':'ZUL', 'isiXhosa':'XHO', 'seTswana':'SET', 'xiTsonga':'XIT',
       'isiNdebele':'NDE', 'seSotho':'SES', 'Other':'OTH', 'siSwati':'SWA', 'Sesotho':'SES'})


**Create Learner Socio Economic Status**

In [14]:
cohort_df["learner_socioeconomic"] = "01"

**Create Learner Disability**

In [15]:
cohort_df['Disability'] = cohort_df['Disability'].fillna('Uknown')
cohort_df['learner_disability'] = cohort_df['Disability'].map({'No':'N', 'no':'N', 'Yes':'09', 'None':'N', 'yes':'09', 'Yes - specwearer':'01', 'N':'N', 'ADD':'09',
       'Visual impairment - spec wearer':'02', 'Y - specwearer':'02', 'Visual impairment - Spec wearer -':'02', 'Uknown':'U', 'Aspergers / Autistic':'06',
       'Patella Alta':'04', 'yes - specwearer':'02', '\\N':'N', 'Y- specwearer':'02'})


**Create Province Code**

In [16]:
cohort_df['learner_province'] = cohort_df['Municipality'].map({'Cape Town Metro':'2', 'Johannesburg Metro':'8', 'Ekurhuleni Metro':'8',
       'Stellenbosch':'2', 'Tshwane Metro':'8', 'Mogale City':'8', 'Blouberg':'10',
       'Midvaal':'8', 'Drakenstein':'2', 'UMlazi municipality':'6', 'Molopo':'7',
       'Overberg':'2', 'Maluti a Phofung':'5', 'West Coast':'2', 'Foreign': '1'})


**Create Learner Designation**

In [17]:
cohort_df["learner_designation"] = "Learner"

**Create Learner Company**

In [18]:
cohort_df['learner_company'] = 'WeThinkCode_'

**Create Learner Title**

In [19]:
cohort_df['learner_title'] = ['Mr.' if gender.lower() == 'male' else 'Ms.' for gender in cohort_df['Gender']]

**Create Learner Contact**

In [20]:
cohort_df['learner_contact'] = 'Larissa Brand'

**Create Learner Contact Phone**

In [21]:
cohort_df['learner_contact_phone'] = '082 774 9982'

**Fix Learner Phone**

In [22]:
def convert_phone_to_string(target_df, column):
    """
    Converts phone number column to a string type.
    
    Args:
        target_df: The dataframe containing the column to be converted.
        column: The column to be converted.
        
    Returns:
        N/A
    """
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
    #convert to float
    target_df[column] = target_df[column].astype(float)
    
    #convert to int
    target_df[column] = target_df[column].astype(int)
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
#convert phone to string
convert_phone_to_string(cohort_df,"Mobile Number")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

**Rename Columns**

In [23]:
cohort_df = cohort_df.rename(columns={"ID Number/ Passport Number":"learner_id", "Firstname":"learner_name", "Surname":"learner_surname", "Birth Date":"learner_birthdate",
                            "Gender":"learner_gender", "Home Address":"learner_postal_address", "Physical Address while studying":"learner_home_address",
                            "Mobile Number":"learner_phone", "Student Email":"learner_email"})


**Subset Columns**

In [24]:
#subset columns
column_name = ["learner_name", "learner_surname",'learner_fullname', 'learner_initials',
               'learner_passport','learner_equity', 'learner_nationality', 'learner_citizenship',
               'learner_language', 'learner_socioeconomic', 'learner_disability','learner_province', 'learner_designation',
              "learner_id", "learner_company", "learner_title", "learner_birthdate", "learner_gender", "learner_postal_address",
              "learner_home_address", "learner_phone", "learner_email", "learner_contact", "learner_contact_phone", "Campus"]

cohort_df = cohort_df[column_name]

**Create Assessor Full Name**

In [25]:
#assessor_df['assessor_fullname'] = assessor_df['Assessor Name'] + ' ' + assessor_df['Assessor Surname']

**Assessor Column Names**

In [26]:
#assessor_df.columns = ['Timestamp', 'assessor_name', 'assessor_surname',
#      'assessor_registration_number', 'Campus', 'assessor_email', 'assessor_fullname']

**Drop Timestamp**

In [27]:
#assessor_df = assessor_df.drop('Timestamp',1)

**Merge cohort data with assessor data**

In [28]:
#cohort_df = cohort_df.merge(assessor_df, on='Campus')

**Subselect Students**

In [29]:
#cohort_df = cohort_df[cohort_df['learner_email'].isin(['awood@student.wethinkcode.co.za'])]

In [30]:
#cohort_df['learner_province']

***Send POE Instructions Document***

In [31]:
#create target columns
#poe_instructions_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_company', 'learner_email']

#create template column
#cohort_df['poe_instructions_template_column'] = "feb67241-52f8-456c-917e-bdd0526619ee"

#send poe instructions through signrequests in bulk
#send_bulk_signrequest(cohort_df, poe_instructions_columns, 'poe_instructions_template_column')

**Send Learner Enrolment Contract Document**

In [32]:
#create target columns
#learner_contract_columns = ['learner_title', 'learner_fullname', 'learner_name', 'learner_surname', 'learner_birthdate', 'learner_id', 'learner_passport',
#                           'learner_equity', 'learner_nationality', 'learner_gender', 'learner_citizenship', 'learner_language',
#                           'learner_socioeconomic', 'learner_disability', 'learner_home_address', 'learner_postal_address',
#                           'learner_province', 'learner_phone', 'learner_email', 'learner_company', 'learner_contact', 'learner_contact_phone',
#                           'learner_designation']

#create template column
#cohort_df['learner_contract_template_column'] = '900be1ba-0b96-4386-a904-fb67069f168e'

#send learner contract through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_contract_columns, 'learner_contract_template_column')


**Send Learner Appeals Procedure**

In [33]:

#create target columns
#learner_appeals_procedure_columns = ['learner_fullname', 'learner_initials']

#create template column
#cohort_df['learner_assessment_contract_template_column'] = '589d226b-1c7e-441f-b99e-1d30a2d37761'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_appeals_procedure_columns, 'learner_assessment_contract_template_column')

**Send Learner Assessment Contract JHB**

In [34]:
#create target columns ****JHB****

#jhb_df = cohort_df[cohort_df['Campus'] == 'Johannesburg']

#learner_assessment_contract_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#jhb_df['learner_assessment_contract_template_column'] = 'b48127ae-d278-467a-89b6-f6b3f010edb4'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(jhb_df, learner_assessment_contract_columns, 'learner_assessment_contract_template_column')

**Send Learner Assessment COntract CPT**

In [35]:
#create target columns ****CPT****

#cpt_df = cohort_df[cohort_df['Campus'] == 'Cape Town']

#learner_assessment_contract_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#cpt_df['learner_assessment_contract_template_column'] = '9a6b58fc-a6f6-4fba-8508-4ef859d98088'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(cpt_df, learner_assessment_contract_columns, 'learner_assessment_contract_template_column')

**Send Learner Declaration of Authenticity**

In [42]:
#create target columns
#learner_authenticity_declaration_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_email']

#create template column
#cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'learner_authenticity_declaration_template_column'] = 'd2308911-b247-4957-b865-e76f504ef91f'
#cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'learner_authenticity_declaration_template_column'] = '78c858be-3acf-4c37-987d-672536c3196e'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_authenticity_declaration_columns, 'learner_authenticity_declaration_template_column')


Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


**Send Learner PreAssessment Meeting Document JHB**

In [37]:
#create target columns  ****JHB****
#learner_preassessment_document_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#jhb_df['learner_preassessment_document_template_column'] = '2cec54b7-0ce3-4bd2-a9c5-29badf20d4f4'

#send learner preassessment meeting document through signrequests in bulk
#send_bulk_signrequest(jhb_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')


**Send Learner PreAssessment Meeting Document CPT**

In [38]:
#create target columns  ****JHB****
#learner_preassessment_document_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
#cpt_df['learner_preassessment_document_template_column'] = '83797d10-f3d4-409d-aa39-93565ccd60fd'

#send learner preassessment meeting document through signrequests in bulk
#send_bulk_signrequest(cpt_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')


**Send Learner Program Evaluation**

In [43]:
#create target columns
#learner_program_evaluation_columns = ['learner_fullname', 'learner_email']

#create template column
#cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'learner_program_evaluation_template_column'] = 'ecae0443-8f3c-44dd-b2ee-7194a4fde67b'
#cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'learner_program_evaluation_template_column'] = 'f97eb0a8-97a9-40c7-84df-29aaee7e9893'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(cohort_df, learner_program_evaluation_columns, 'learner_program_evaluation_template_column')


Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201
Signer: mufaro@thoughtquest.co.za , Status: 201


***Create Test Dataframe***

In [39]:
cohort_df = cohort_df.groupby(['Campus']).head(1).groupby('Campus').head(1).copy()
cohort_df['learner_email'] = 'mufaro@thoughtquest.co.za'
#send_bulk_signrequest(test_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')


3